In [82]:
import os
from os.path import join as oj
import sys, time
sys.path.insert(1, oj(sys.path[0], '..'))  # insert parent path
import seaborn as sns
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from copy import deepcopy
import pickle as pkl
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import cross_validate, ShuffleSplit, train_test_split
import pandas as pd
import data_pecarn 
import data_psrc
import data
import util
from style import *
from collections import Counter
from data import feats_numerical, feats_categorical, meta, outcome_def

# plt.style.use('dark_background')
# sns.set(style="white")
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# benchmark bivariable associations

In [100]:
df_pecarn, df_psrc, common_feats, filtered_feats_pecarn, filtered_feats_psrc = data.load_it_all(dummy=False, impute=False, frac_missing_allowed=1)
d = df_pecarn # [common_feats]
d['Age<2'] = d['Age'] < 2
# data.select_final_feats(common_feats)

/accounts/projects/vision/.local/lib/python3.7/site-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [106]:
# can also check vars that were not used
rules = [
    ('AbdTrauma_or_SeatBeltSign', ['yes']),
    ('GCSScore', range(14)),
#     ('AbdTenderDegree', ['Mild', 'Moderate', 'Severe']),
    ('AbdTenderDegree', ['None']),    
    ('AbdTenderDegree', ['Mild']),    
    ('AbdTenderDegree', ['Moderate']),    
    ('AbdTenderDegree', ['Severe']),        
    ('ThoracicTrauma', ['yes']),        
    ('AbdomenPain', ['yes']),
    ('DecrBreathSound', ['yes']),
    ('VomitWretch', ['yes']),
    
    # not used in rule
    ('Age<2', [True]),    
    ('AbdDistention', ['yes']),
    ('MOI', ['Motor vehicle collision']),
    ('DistractingPain', ['yes']),    
    ('CostalTender', ['yes']),
    ('ThoracicTender', ['yes']),    
    ('Hypotension', ['yes']),        
    ('AbdomenTender', [1]),    
]
index = ['NA', 'IAIw/I', 'No IAIw/I', 'Difference']
o = data.outcome_def
r = {(k, v[0]) : [] for k, v in rules}
io = d[o] == 1
print('num IwI and key / num IwI (not nan) \t num without IwI and key / num without IwI (not nane) ')
for rule in rules:
    k, vals = rule
    kk = (k, vals[0])
    ik = d[k].isin(vals) # d[o]==1
    inan = d[k].isna() | (d[k] == 'unknown')
    r[kk].append(np.sum(inan))   
    num1 = np.sum(d[ik][o]==1)
    denom1 = (io & ~inan).sum()
    r[kk].append(f'{num1}/{denom1}')
    num2 = np.sum(d[ik][o]==0)
    denom2 = (~io & ~inan).sum()
    r[kk].append(f'{num2}/{denom2}')
    r[kk].append(f'{num1/denom1 - num2/denom2:0.3f}')    
r = pd.DataFrame.from_dict(r)
r.index = index
r.transpose()

num IwI and key / num IwI (not nan) 	 num without IwI and key / num without IwI (not nane) 


NA   IAIw/I   No IAIw/I  \
AbdTrauma_or_SeatBeltSign yes                         0  112/203  1851/11841   
GCSScore                  0                           4   67/203   928/11837   
AbdTenderDegree           None                     8447    0/113      0/3484   
                          Mild                     8447   12/113   1623/3484   
                          Moderate                 8447   42/113   1501/3484   
                          Severe                   8447   59/113    360/3484   
ThoracicTrauma            yes                        76   66/201  1976/11767   
AbdomenPain               yes                       148  105/201  3515/11695   
DecrBreathSound           yes                       233   25/197   231/11614   
VomitWretch               yes                       472   49/165  1104/11407   
Age<2                     True                        0   10/203  1157/11841   
AbdDistention             yes                       528   49/199   228/11317   
MOI                       Motor vehicle collision   476   91/194  3741/11374   
DistractingPain           yes                       889   61/159  2744/10996   
CostalTender              yes                         0   41/203  1324/11841   
ThoracicTender            yes                      1203   36/136  1886/10705   
Hypotension               yes                         0   24/203   150/11841   
AbdomenTender             True                     8476    0/113      0/3455   

                                                  Difference  
AbdTrauma_or_SeatBeltSign yes                          0.395  
GCSScore                  0                            0.252  
AbdTenderDegree           None                         0.000  
                          Mild                        -0.360  
                          Moderate                    -0.059  
                          Severe                       0.419  
ThoracicTrauma            yes                          0.160  
AbdomenPain               yes                          0.222  
DecrBreathSound           yes                          0.107  
VomitWretch               yes                          0.200  
Age<2                     True                        -0.048  
AbdDistention             yes                          0.226  
MOI                       Motor vehicle collision      0.140  
DistractingPain           yes                          0.134  
CostalTender              yes                          0.090  
ThoracicTender            yes                          0.089  
Hypotension               yes                          0.106  
AbdomenTender             True                         0.000

In [101]:
# Abdominal tenderness degree, 
# Surrogate: Abdominal tenderness, abdominal pain, costal tenderness, left costal tenderness
# Missing 205

# 1 is yes, 2 is no, 3 unable to asses, 4 didn't answer
np.unique(df_pecarn['AbdomenTender'], return_counts=True) 

(array([1., 2., 3., 4.]), array([3611, 7236, 1035,  162]))

In [102]:
np.unique(df_pecarn['AbdTenderDegree'], return_counts=True)
# sums should be 1613, 1536, 419, 205 (although 205 is only after filling in a bunch)

(array(['Mild', 'Moderate', 'Severe', 'unknown'], dtype=object),
 array([1635, 1543,  419, 8447]))

In [103]:
a = deepcopy(df_pecarn['AbdTenderDegree'].values)
a[df_pecarn['AbdomenTender'].isin([2, 3, 4])] = 'unknown'
# a.loc[df_pecarn['AbdomenTender'].isin([1])] = 'test'
# idxs_combined = ((df_pecarn['AbdTenderDegrenee']=='unknown') \
#                  & (df_pecarn['AbdomenTender'].isin([2])))


# re set the values
df_pecarn['AbdTenderDegree'] = a
d = df_pecarn
# if abdoment tender is false then set tender degree to none
# df_pecarn.loc[df_pecarn['AbdomenTender'] == 2, 'AbdTenderDegree'] = 'None'

In [104]:
np.unique(a, return_counts=True)
# sums should be 1613, 1536, 419, 205 (although 205 is only after filling in a bunch)

(array(['Mild', 'Moderate', 'Severe', 'unknown'], dtype=object),
 array([1613, 1536,  419, 8476]))

In [121]:
np.unique(d['AbdTenderDegree'] , return_counts=True)

(array(['Mild', 'Moderate', 'Severe', 'unknown'], dtype=object),
 array([1613, 1536,  419, 8476]))

# recreate / evaluate rule

In [122]:
def pecarn_rule_predict(d, o=data.outcome_def):
    n = d.shape[0]
    npos = d[o].sum()
    print(f'{"Initial":<25} {npos} / {n}')
    risks = np.array([np.nan] * d.shape[0])
    rules = [
        ('AbdTrauma_or_SeatBeltSign', ['yes']),
        ('GCSScore', range(14)),
        ('AbdTenderDegree', ['Mild', 'Moderate', 'Severe']),
        ('ThoracicTrauma', ['yes']),        
        ('AbdomenPain', ['yes']),
        ('DecrBreathSound', ['yes']),
        ('VomitWretch', ['yes']),
    ]
    for rule in rules:
        k, vals = rule
        idxs = d[k].isin(vals)
        do = d[idxs]
        d = d[~idxs]
        num2 = do[o].sum()
        denom2 = do.shape[0]
        print(f'{k:<25} {d[o].sum():>3} / {d.shape[0]:>5}\t{num2:>3} / {denom2:>4} ({num2/denom2*100:0.1f})')
    
    low_risk_patients = d
    patients_missed = low_risk_patients[low_risk_patients[o] == 1]
    
    # calc metrics
    fn = patients_missed.shape[0]
    tp = npos - fn
    tn = low_risk_patients.shape[0] - fn
    fp = (n - low_risk_patients.shape[0]) - tp
    stats = {
        'fn': fn,
        'tp': tp,
        'tn': tn,
        'fp': fp,
        'sensitivity': tp / npos * 100,
        'specificity': tn / (tn + fp) * 100.
    }
    return d, patients_missed, stats

low_risk_patients, missed_patients, stats = pecarn_rule_predict(df_pecarn)
for stat in ['sensitivity', 'specificity']:
    print(stat, f'{stats[stat]:0.2f}')
missed_patients[['Age', 'Sex', 'MOI']]
assert missed_patients.shape[0] == 6, 'should find 6 low-risk patients'

# 203 / 12044
#  91 / 10081
#  53 /  9255
#  17 /  6723

Initial                   203 / 12044
AbdTrauma_or_SeatBeltSign  91 / 10081	112 / 1963 (5.7)
GCSScore                   53 /  9255	 38 /  826 (4.6)
AbdTenderDegree            17 /  6767	 36 / 2488 (1.4)
ThoracicTrauma             11 /  5805	  6 /  962 (0.6)
AbdomenPain                 9 /  5469	  2 /  336 (0.6)
DecrBreathSound             8 /  5435	  1 /   34 (2.9)
VomitWretch                 6 /  5040	  2 /  395 (0.5)
sensitivity 97.04
specificity 42.51


In [22]:
low_risk_patients, missed_patients, stats = pecarn_rule_predict(df_psrc)
for stat in ['sensitivity', 'specificity']:
    print(stat, f'{stats[stat]:0.2f}')
missed_patients[['Age', 'MOI']]

Initial                   62 / 2188
AbdTrauma_or_SeatBeltSign  18 /  1708	 44 /  480 (9.2)
GCSScore                    8 /  1441	 10 /  267 (3.7)
AbdTenderDegree             2 /  1216	  6 /  225 (2.7)
ThoracicTrauma              2 /  1123	  0 /   93 (0.0)
AbdomenPain                 2 /   993	  0 /  130 (0.0)
DecrBreathSound             2 /   993	  0 /    0 (nan)
VomitWretch                 2 /   937	  0 /   56 (0.0)
sensitivity 96.77
specificity 43.98


/accounts/projects/vision/.local/lib/python3.7/site-packages/ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in long_scalars


,Age,MOI
1164,1.0,Motor vehicle collision
1755,6.0,Fall from an elevation


In [11]:
df_psrc.loc[missed_patients.index].to_csv('processed/missed_patients_psrc.csv')